In [129]:
#pip install psycopg2
import psycopg2
import numpy as np
from langchain_core.document_loaders import BlobLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langgraph_sdk.schema import IfNotExists
from matplotlib.backend_bases import cursors
from matplotlib.pyplot import close
from openai import embeddings
from pyspark.sql.connect.functions import length
from sqlalchemy import RollbackToSavepointClause
from sqlalchemy.dialects.mssql.information_schema import columns
from sqlalchemy.sql.roles import ColumnArgumentRole

ImportError: cannot import name 'length' from 'pyspark.sql.connect.functions' (/Users/gyanapradhan/PyCharmMiscProject/.venv/lib/python3.13/site-packages/pyspark/sql/connect/functions/__init__.py)

In [104]:
#pip install -qU  langchain-google-genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [193]:
import os
#ca_cert_path = os.path.join(os.getcwd(), 'ca.pem')
ca_cert_path='/Users/gyanapradhan/Desktop/Datasets/ca.pem'
#


DB_USER = '*********'
DB_PASSWORD = '*********'
DB_HOST = 'pg-4***********.f.******cloud.com'
DB_PORT = '19807'
DB_NAME = 'defaultdb'

# con_string ='''postgres://postgres::********_****-4**8-grpradh******-*d10.f.*****cloud.com:19807/defaultdb?sslmode='verify-ca',sslrootcert=~/Desktop/Datasets/ca.pem'''
# connection =psycopg2.connect(con_string)

connection = psycopg2.connect(
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT,
    database=DB_NAME,
    sslmode='verify-ca', # Use 'verify-full' for maximum security
    sslrootcert=ca_cert_path
)

cursor= connection.cursor()

In [125]:
cursor.execute("SELECT VERSION()")
version = cursor.fetchone()[0]
print(f"PostgreSQL version: {version}")




PostgreSQL version: PostgreSQL 17.6 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 14.2.1 20240912 (Red Hat 14.2.1-3), 64-bit


In [24]:
# stmt_sql='''CREATE EXTENSION vector;
# '''
# cursor.execute(stmt_sql)


In [ ]:
# cursor.execute("ROLLBACK;")
# cursor.close()
# connection.close()


In [126]:
# query_sql ='''
# CREATE TABLE IF NOT EXISTS test(
#     id BIGSERIAL PRIMARY KEY,
#     text TEXT,
# );
# '''

cursor.execute("CREATE TABLE IF NOT EXISTS test(id BIGSERIAL PRIMARY KEY,text TEXT,embedding VECTOR(1536));")



In [ ]:
Query_SQL2= '''
CREATE TABLE IF NOT EXISTS myvector(
id bigserial PRIMARY KEY,
text TEXT,
embedding vector(3));
'''
cursor.execute(Query_SQL2)

In [49]:
query_sql3='''insert into myvector (text, embedding) values
    ('Apple', '[1.0, 2.0, 3.0]'),
    ('Banana', '[1.1, 2.2, 3.3]'),
    ('Orange', '[0.9, 1.9, 2.9]'),
    ('Carrot', '[5.0, 6.0, 7.0]');
'''
cursor.execute(query_sql3)
cursor.execute('COMMIT')



In [164]:
print (cursor.execute('SELECT * FROM carvector;'))
rows = cursor.fetchall()
for row  in rows:
    print(row[0], row[1], row[2])

None
1 AC,ACE,1 Gen,1993,Cabriolet,3.5 MT,Cabriolet [-0.00071896036,0.0071785697,0.017956343,-0.046055865,-0.006209309,-0.0046618036,-0.007986733,0.008863689,-0.011252771,0.0058344845,-0.007802792,-0.012074441,0.0032891172,0.03154514,0.13510631,-0.020477526,0.019032141,0.009570249,-0.011889939,-0.022505712,0.02299799,-0.005551252,0.013018448,0.016267344,-0.008576406,0.00034837658,0.01801353,0.024447083,0.031674534,0.013654328,0.012034375,0.0145986825,0.0016210042,0.012115691,-0.006205006,0.0155430045,0.029474976,-0.0039039887,0.013102706,0.014212371,0.0020736428,0.016390972,-0.0037892212,0.0071730833,-0.0058263466,-0.016433919,0.007453289,-0.008461985,-0.005941287,0.029595165,-0.0030761864,-0.006548981,0.021802144,-0.204514,-0.0034124348,0.0006031471,0.0102536585,0.008576007,0.011066207,-0.015786545,0.0025803912,0.0064375186,-0.0019660369,-0.053426508,-0.004855106,-0.0020317943,-0.0072425925,0.005434216,-0.008846116,-0.0276037,0.0005321342,-0.006068601,-0.00967321,-0.0128283035,0.02469

In [130]:
cursor.execute("select id,text,embedding from myvector order by embedding <-> '[1.0, 2.0, 3.0]' limit 5;")
rows1= cursor.fetchall()
for row in rows1:
    print(row[0], row[1], row[2])


9 Apple [1,2,3]
5 Apple [1,2,3]
1 Apple [1,2,3]
3 Orange [0.9,1.9,2.9]
7 Orange [0.9,1.9,2.9]


In [ ]:
# os.environ['OPENAI_API_KEY'] = open('secret key').read()

In [144]:
Query_SQL3= '''
CREATE TABLE IF NOT EXISTS carvector(
id bigserial PRIMARY KEY,
text TEXT,
embedding vector(3072));
'''
cursor.execute(Query_SQL3)
connection.commit()


In [153]:
texts= [
"AC,ACE,1 Gen,1993,Cabriolet,3.5 MT,Cabriolet",
"AC,ACE,1 Gen,1993,Roadster,4.9 AT",
"Acura,CL,1 Gen,1996,Coupe,2.2 AT,Coupe",
"Acura,CSX,1 Gen,2006,Sedan,2.0 AT,Sedan",
"Acura,CSX,1 Gen,2006,Sedan,2.0 MT,Sedan",
"Adler,Diplomat,1 Gen,1934,Sedan Pullman",
"Adler,Trumpf Junior,1 Gen,1934,Sedan",
"Alfa Romeo,105/115,1 Gen,1965,Coupe",
"Alfa Romeo,145,1 Gen,1994,Hatchback",
"Alpina,B3,E36,1993,Coupe,3.0 MT",
"Alpina,B3,E36,1993,Coupe,3.2 AT",
]

In [107]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# embeddings= OpenAIEmbeddings(model= "gpt-oss-20b")
# embedding_list =[]
# for text in texts:
#     embedding_list.append(embeddings.embed_query(text))



In [154]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
embedding_list =[]
for text in texts:
    embedding_list.append(embeddings.embed_query(text))

In [ ]:
print(embedding_list)

In [118]:
len(embedding_list[0])

3072

In [160]:
for i in range(len(embedding_list)):
    content = texts[i]
    embedding=embedding_list[i]
    cursor.execute("INSERT INTO carvector (text,embedding) values(%s, %s);", (content, embedding))
connection.commit()





In [196]:
text_search='Cabriolet v3.2'
embedding_sr = embeddings.embed_query(text_search)

In [197]:
search_qry= 'Select id ,text from carvector order by embedding <-> %s::vector Limit 5;'
cursor.execute(search_qry,(embedding_sr,))

In [198]:
result= cursor.fetchall()
for row in result:
    print(row)


(1, 'AC,ACE,1 Gen,1993,Cabriolet,3.5 MT,Cabriolet')
(11, 'Alpina,B3,E36,1993,Coupe,3.2 AT')
(2, 'AC,ACE,1 Gen,1993,Roadster,4.9 AT')
(10, 'Alpina,B3,E36,1993,Coupe,3.0 MT')
(3, 'Acura,CL,1 Gen,1996,Coupe,2.2 AT,Coupe')


(11, 'Alpina,B3,E36,1993,Coupe,3.2 AT')
(10, 'Alpina,B3,E36,1993,Coupe,3.0 MT')
(1, 'AC,ACE,1 Gen,1993,Cabriolet,3.5 MT,Cabriolet')
(4, 'Acura,CSX,1 Gen,2006,Sedan,2.0 AT,Sedan')
(3, 'Acura,CL,1 Gen,1996,Coupe,2.2 AT,Coupe')
